# Quickstart

https://www.django-rest-framework.org/tutorial/quickstart/

## Project setup
- startproject is run with a . after the project name to put the app in the project directory, but it helps avoid clashes with external module's namespaces

## Serializers
`HyperlinkedModelSerializer` used instead of primary key and various other relationships - hyperlinking is good RESTful design

## Views
We group common behavior into classes called `ViewSets`.  These can be very concise and organized.

## URLs
When using viewsets, we can generate the URL conf for our API by registering viewsets with a router class in `urls.py`
- if we need more control over API URLs we can use class-based views and explicitely write URL conf
- default login and logout views are added for browsable API

## Pagination
Determines how many objects per page are returned.  Edit `settings.py`.

# API access
Run the server, then 
`curl -H 'Accept: application/json; indent=4' -u admin:password123 http://127.0.0.1:8000/users/`
or with HTTPie
`http -a admin:password123 http://127.0.0.1:8000/users/`
or point the browser to
http://127.0.0.1:8000/users/

# In-depth Tutorial
https://www.django-rest-framework.org/tutorial/1-serialization/